In [1]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
import nltk
import pickle
nltk.download('punkt')
from tqdm import tqdm

#from nltk.tokenize import word_tokenize
#from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import time
from datetime import timedelta

2024-12-12 12:06:36.851537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-12 12:06:36.869648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-12 12:06:36.875081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 12:06:36.889426: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 12:06:38.280615: W tensorflow/compiler/tf2

In [2]:
print(np.__version__)
print(torch.__version__)

2.0.1
1.13.0+cu117


In [3]:
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    print(f"CUDA version: {torch.version.cuda}")

num_gpus = torch.cuda.device_count()
print(f"Number of GPUs: {num_gpus}")

if num_gpus > 0:
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA available: True
CUDA version: 11.7
Number of GPUs: 1
GPU 0: NVIDIA A100-SXM4-80GB


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
from torchtext.vocab import GloVe
glove = GloVe(name='twitter.27B', dim=200)

In [6]:
import pickle

with open('/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/Dataset Multilingual/multilingual_keras_tokenizer.pickle', 'rb') as handle:
    keras_tokenizer = pickle.load(handle)
    
vocab_size = len(keras_tokenizer.word_index) + 1
print('Vocab size:', vocab_size)

Vocab size: 8771


In [7]:
df = pd.read_excel('/home/basilmusyaffa19/Skripsi Basil/Dataset/FIX/clean_multilingual_22112024.xlsx', engine='openpyxl')
df

,teks,label
0,babe go day sip cappuccino think love send kis...,1
1,sy dr rimbawan dirmawa ipb hormat jajang roni ...,1
2,send logo ur lover name joined heart txt love ...,1
3,fine difficulty phone work mine pls send anoth...,1
4,already sabarish asked go,1
...,...,...
6131,freemsg today day ready horny live town love s...,1
6132,selamat dapat voucher zalora rp ribu masuk kod...,1
6133,terima kasih hasil laku isi pulsa smartfren ma...,1
6134,pangmeulikeun heula kredit bpa simpati ribu ga...,1


In [8]:
print("Jumlah data sebelum:", len(df))

# Menghapus data kosong
df = df.replace('', np.nan).dropna()
# Hapus NaN
df = df.dropna(subset=['teks'])
# Menghapus nilai float
df = df[~df['teks'].apply(lambda x: isinstance(x, float))]
# Menghapus semua baris yang duplikat
df = df.drop_duplicates(subset=['teks'], keep='first')

print("Jumlah data setelah:", len(df))

Jumlah data sebelum: 6136
Jumlah data setelah: 6136


In [9]:
def check_coverage(vocab_list, glove):
    embedding_words = set(glove.itos)
    num_words_found = 0
    oov = []

    for word in tqdm(vocab_list):
        if word in embedding_words:
            num_words_found += 1
        else:
            oov.append(word)

    total_words = len(vocab_list)
    print('Found embeddings for {:.2%} of vocab'.format(num_words_found / total_words))
    print('Number of words not found: {}'.format(len(oov)))
    
    return oov

In [10]:
vocab_list = list(keras_tokenizer.word_index.keys())
oov = check_coverage(vocab_list, glove)

print("\nBeberapa kata yang tidak ditemukan:")
for word in oov[:10]:
    print(f"'{word}'")
    
df_oov = pd.DataFrame(oov, columns=['Kata OOV'])
df_oov.to_excel('/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/kata_oov_multilingual.xlsx', index=False)

100%|██████████| 8770/8770 [00:00<00:00, 516129.68it/s]


Found embeddings for 88.61% of vocab
Number of words not found: 999

Beberapa kata yang tidak ditemukan:
'tcash'
'chikku'
'freemsg'
'rakhesh'
'pobox'
'indosatooredoo'
'savamob'
'bronet'
'tronik'
'mytelkomsel'


In [11]:
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))
embedding_matrix.shape

(8771, 200)

In [12]:
def initialize_embedding_matrix(tokenizer, oov_words, embedding_matrix, model, embedding_dim=200):
    start_time = time.time()
    word_embeddings = {}
    
    for word, i in tqdm(tokenizer.word_index.items(), desc="Creating embedding matrix"): 
        if word in oov_words:
            random_vector = np.random.uniform(-0.25, 0.25, embedding_dim)
            random_vector = random_vector / np.linalg.norm(random_vector) * np.sqrt(embedding_dim)
            embedding_matrix[i] = random_vector
            word_embeddings[i] = (word, random_vector)
        else:  
            embedding_vector = model[word].numpy()
            embedding_matrix[i] = embedding_vector
            word_embeddings[i] = (word, embedding_vector)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\nTotal waktu pembuatan embedding matrix: {elapsed_time}s")
    
    return embedding_matrix, word_embeddings

In [13]:
embedding_matrix_glove, word_embeddings_glove = initialize_embedding_matrix(
    keras_tokenizer,
    oov,
    embedding_matrix,
    glove
)

Creating embedding matrix: 100%|██████████| 8770/8770 [00:00<00:00, 16296.59it/s]


Total waktu pembuatan embedding matrix: 0.547499418258667s


In [14]:
embedding_matrix_glove

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.36776   ,  0.62058002, -0.16424   , ..., -0.31200001,
        -0.02699   ,  0.084279  ],
       [-0.20837   ,  0.37200999, -0.066035  , ..., -0.0091251 ,
        -0.26668999,  0.29602   ],
       ...,
       [-0.075862  ,  0.1478    , -0.18271001, ..., -0.16144   ,
         0.12695999, -0.67714   ],
       [-0.24065415, -1.64383852, -1.16437044, ..., -1.33313845,
        -1.59457131,  1.47274856],
       [-1.32298282,  0.1102973 , -0.06383698, ..., -0.63275833,
         0.46027717,  0.77345273]])

In [15]:
for idx, (word, vector) in list(word_embeddings_glove.items())[:5]:
    print(f"Index: {idx}")
    print(f"Word: {word}")
    vector_np = np.array(vector) if not isinstance(vector, np.ndarray) else vector
    print(f"Vector (first 10 values): {vector_np[:10]}")
    print("---")

Index: 1
Word: call
Vector (first 10 values): [-0.36776   0.62058  -0.16424  -0.55841   0.19918  -0.021146  0.70505
  0.27204   0.53151  -0.3686  ]
---
Index: 2
Word: get
Vector (first 10 values): [-0.20837   0.37201  -0.066035 -0.44603   0.37284  -0.12545   0.86787
  0.4099    0.23244   0.3407  ]
---
Index: 3
Word: ur
Vector (first 10 values): [ 0.21273   0.11611   0.16871  -0.35413   0.13989  -0.37088   0.78829
  0.008818 -0.048655  0.65067 ]
---
Index: 4
Word: go
Vector (first 10 values): [ 0.24313   0.011597  0.38135  -0.5835    0.14441   0.56247   0.52358
  0.20859   0.3643    0.16414 ]
---
Index: 5
Word: ok
Vector (first 10 values): [-0.039412 -0.39475  -0.34866   0.14839  -0.56289   0.20934   0.35344
  0.16257   0.027069  0.27543 ]
---


In [16]:
target_word = "chikku"

word_found = False
for idx, (word, vector) in word_embeddings_glove.items():
    if word == target_word:
        print(f"Index: {idx}")
        print(f"Word: {word}")
        print(f"Vector: {vector}")
        word_found = True
        break

if not word_found:
    print(f"Kata '{target_word}' tidak ditemukan")

Index: 531
Word: chikku
Vector: [-0.88899276 -0.73291837  0.60996541 -0.79552452  0.01565385  0.86605629
  0.601596   -0.72080997 -1.1569533  -1.02313873 -0.01146402  0.30010013
 -0.36260153 -1.17950303  0.12731209 -0.27679852 -1.53906447 -0.68430769
 -0.92459931 -0.79644716 -1.2356532  -1.37369282  0.65013007 -1.26218849
  1.68260395  0.25148757  0.44901461 -0.07733233  1.35351462  1.37775557
 -0.18565755  0.73124169  1.71877217  1.31947088  1.26259106  1.71532353
 -0.5110958  -0.19901703  0.43186352  1.42772439  0.58080063 -1.45906192
  0.18456078 -0.71246544  0.26679146  0.77731645  0.49631838 -0.28769978
 -0.6189252   1.64811938 -0.20862224 -0.55636232 -1.62189047  0.13780929
  1.52216887  0.59552742  1.59737255  1.25341108  0.36921556  1.16894229
 -0.02247133 -1.2947174   0.91390108 -1.27624297 -0.2760889   0.46732463
 -1.56917048  0.32893229 -0.92997065  0.8475611  -0.48986531  0.36904913
  0.44480049 -0.00896775 -0.36865587 -0.62337652  0.94076459  1.24863189
 -1.70111327 -0.940

In [17]:
path = '/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/Dataset Multilingual/Hasil Embedding/21 Nov/embedding_matrix_gloveTwitter_200D_21112024.npy'
np.save(path, embedding_matrix_glove)

In [18]:
import os
file_size = os.path.getsize('/home/basilmusyaffa19/Skripsi Basil/Embedding Matrix/Dataset Multilingual/Hasil Embedding/21 Nov/embedding_matrix_gloveTwitter_200D_21112024.npy')
print(f"Ukuran file: {file_size/1024/1024:.2f} MB")

Ukuran file: 13.38 MB
